In [34]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.datasets import imdb
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [17]:
from pandas import read_csv
df = pd.read_csv('sample.csv',encoding='latin-1', header=0,  sep = ',', names = ['label', 'content'], error_bad_lines=False)
df.head(3)

,label,content
0,Bachelors Degree,A
1,Bachelors Degree,B
2,Bachelors Degree,S


In [18]:
#cleaning


def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

#df= df.dropna()
df[df["label"].apply(lambda x: x.isnumeric())] #dropping null values
df[df.label.apply(lambda x: x !="")] #filtering out rows with non-numeric characters in the "label" column
df[df.content.apply(lambda x: x !="")] #filterin out rows with empty comments
df.head(3)

,label,content
0,Bachelors Degree,A
1,Bachelors Degree,B
2,Bachelors Degree,S


In [39]:
#Tokenize with padding 

def get_sequence_of_tokens(x):
    ## tokenization
    tokenizer.fit_on_texts(x)
    total_words = len(tokenizer.word_index) + 1
    
    # Encode training data sentences into sequences
    input_sequences = tokenizer.texts_to_sequences(x)    
    return input_sequences, total_words


sequences,num_words  = get_sequence_of_tokens(df['content'])

print("num_words= "+str(num_words))
print("maxlen of all sentences= "+ str(maxlen))
print(sequences)

num_words= 66
maxlen of all sentences= 3
[[2], [4], [6], [9], [15], [16], [17], [18], [19], [20], [21], [9, 10], [22], [23], [24], [25], [26], [27], [28], [29], [7, 30], [31], [7, 11], [32], [33], [34], [3, 35], [3, 36], [3, 37], [3, 38], [3, 39], [1, 2, 8], [1, 2, 6], [1, 2, 40], [1, 41], [1, 42], [1, 4, 2], [1, 4, 8], [1, 4, 11], [1, 12, 8], [1, 12, 10], [1, 43, 6], [13, 44], [13, 45], [46, 47], [48, 49], [50, 14], [7, 51], [5, 52], [5, 53], [5, 54], [5, 55], [56, 14], [57], [58], [59], [60, 61], [62], [63], [64, 65]]


In [41]:
#Padding    lenght(pad) is the max number or word in the sentences and each number is one of the word

def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors = input_sequences
    return predictors, max_sequence_len

data, m = generate_padded_sequences(sequences)
print(data)
print(m)

[[ 0  0  2]
 [ 0  0  4]
 [ 0  0  6]
 [ 0  0  9]
 [ 0  0 15]
 [ 0  0 16]
 [ 0  0 17]
 [ 0  0 18]
 [ 0  0 19]
 [ 0  0 20]
 [ 0  0 21]
 [ 0  9 10]
 [ 0  0 22]
 [ 0  0 23]
 [ 0  0 24]
 [ 0  0 25]
 [ 0  0 26]
 [ 0  0 27]
 [ 0  0 28]
 [ 0  0 29]
 [ 0  7 30]
 [ 0  0 31]
 [ 0  7 11]
 [ 0  0 32]
 [ 0  0 33]
 [ 0  0 34]
 [ 0  3 35]
 [ 0  3 36]
 [ 0  3 37]
 [ 0  3 38]
 [ 0  3 39]
 [ 1  2  8]
 [ 1  2  6]
 [ 1  2 40]
 [ 0  1 41]
 [ 0  1 42]
 [ 1  4  2]
 [ 1  4  8]
 [ 1  4 11]
 [ 1 12  8]
 [ 1 12 10]
 [ 1 43  6]
 [ 0 13 44]
 [ 0 13 45]
 [ 0 46 47]
 [ 0 48 49]
 [ 0 50 14]
 [ 0  7 51]
 [ 0  5 52]
 [ 0  5 53]
 [ 0  5 54]
 [ 0  5 55]
 [ 0 56 14]
 [ 0  0 57]
 [ 0  0 58]
 [ 0  0 59]
 [ 0 60 61]
 [ 0  0 62]
 [ 0  0 63]
 [ 0 64 65]]
3


In [32]:
#matrix length(pad) is the number of vocab and each 1 is represent if that word is appear in the sentence or not
encoded_docs = tokenizer.sequences_to_matrix(sequences, mode='binary')
print(encoded_docs[1:4])

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=50)) 
### an embedding layer to expand tokens, allowing NN to represent the word in a meaningful way ###
######Embedding(vocabolary size, dimension of embedding, MAX length of each sentence)####
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) # 
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
outputSize = 100
e = Embedding(num_words, outputSize, input_length=maxlen)
e